# M3 month with exog (prediction of exogenous) Stupid way

In [2]:
import numpy as np
import pyramid
import pandas as pd
import math
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
import requests
from io import BytesIO
from sklearn import metrics
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Legend
import pandas as pd
from bokeh.models import Span

In [14]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [106]:
df = pd.read_excel('M3C.xls',sheet_name='M3Month')
df = df[df.Category == 'MICRO       ']
df_results_Theta = pd.read_excel('M3Forecast.xls',sheet_name='THETA',header=None)
df_results_FPro = pd.read_excel('M3Forecast.xls',sheet_name='ForecastPro',header=None)
df_results_FX = pd.read_excel('M3Forecast.xls',sheet_name='ForcX',header=None)
df.head()

,Series,N,NF,Category,Starting Year,Starting Month,1,2,3,4,...,135,136,137,138,139,140,141,142,143,144
0,N1402,68,18,MICRO,1990,1,2640.0,2640.0,2160.0,4200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N1403,68,18,MICRO,1990,1,1680.0,1920.0,120.0,1080.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N1404,68,18,MICRO,1990,1,1140.0,720.0,4860.0,1200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N1405,68,18,MICRO,1990,1,180.0,940.0,2040.0,800.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N1406,68,18,MICRO,1990,1,2000.0,1550.0,4450.0,3050.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
N_series = 9
N_series_1990 = 15
N = df['N'][N_series]
NF = df['NF'][N_series]
start_year = df['Starting Year'][N_series]
start_month = df['Starting Month'][N_series]
Id = df['Series'][N_series]

K1 = N-12
K2 = N+6
train_data = np.array(df.iloc[N_series,6:K1]).astype(np.int)
test_data = np.array(df.iloc[N_series,K1:K2]).astype(np.int)
exogenous_data = np.array(df.iloc[10:N_series_1990,6:K1]).astype(np.int).T

In [103]:
def generate_exogenous_2(exogenous_data,test_data):
    N_obs = exogenous_data.shape[0]
    K_exog = exogenous_data.shape[1]
    N_pred = test_data.shape[0]
    new_exogenous = np.empty((N_pred,K_exog))
    for k in range(K_exog):
        tt = exogenous_data[:,k]
        fitter = auto_arima(tt,start_p=1, start_q=1, max_p=5, max_q=5, m=12,
                          start_P=0, d=1, max_D=2,max_P = 2, max_Q = 2, trace=False,
                          error_action='ignore', suppress_warnings=True,stepwise=True)
        new_exog = fitter.predict(n_periods=N_pred)
        new_exogenous[:,k] = new_exog
    return new_exogenous  

In [108]:
new_exogenous = generate_exogenous_2(exogenous_data,test_data)

In [107]:
forecast_Theta = np.array(df_results_Theta[df_results_Theta[0]==Id]).reshape(-1)[2:].astype(np.int)
forecast_FPro = np.array(df_results_FPro[df_results_FPro[0]==Id]).reshape(-1)[2:].astype(np.int)
forecast_FX = np.array(df_results_FX[df_results_FX[0]==Id]).reshape(-1)[2:].astype(np.int)
del df_results_FPro
del df_results_FX
del df_results_Theta

In [28]:
output_notebook()

def plot_arima(truth, forecasts, title="ARIMA", xaxis_label='Time',
               yaxis_label='Value', c1='#A6CEE3', c2='#B2DF8A', 
               forecast_start=None, **kwargs):
    
    # make truth and forecasts into pandas series
    n_truth = truth.shape[0]
    n_forecasts = forecasts.shape[0]
    
    # always plot truth the same
    truth = pd.Series(truth, index=np.arange(truth.shape[0]))
    
    # if no defined forecast start, start at the end
    if forecast_start is None:
        idx = np.arange(n_truth, n_truth + n_forecasts)
    else:
        idx = np.arange(forecast_start, n_forecasts)
    forecasts = pd.Series(forecasts, index=idx)
    
    # set up the plot
    p = figure(title=title, plot_height=400, **kwargs)
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = yaxis_label
    
    # add the lines
    p.line(forecasts.index, forecasts.values, color=c2, legend='Forecasted')
    p.line(truth.index, truth.values, color=c1, legend='Observed')
    vline = Span(location=49,dimension='height', line_color='red',line_width=1)
    p.renderers.extend([vline])
    
    return p

def plot_arima_comparison(truth_forecast, forecasts,forecasts_rs,forecast_1,forecast_2,forecast_3,forecast_4, title="Comparison", xaxis_label='Time',
               yaxis_label='Value', c1='#A6CEE3', c2='#B2DF8A', 
               forecast_start=None, **kwargs):
    
    forecasts = pd.Series(forecasts)
    forecasts_rs = pd.Series(forecasts_rs)
    true_forecasts = pd.Series(truth_forecast)
    forecasts_1 = pd.Series(forecast_1)
    forecasts_2 = pd.Series(forecast_2)
    forecasts_3 = pd.Series(forecast_3)
    forecasts_4 = pd.Series(forecast_4)
    
    # set up the plot
    p = figure(title=title, plot_height=600, plot_width=1000, **kwargs)
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = yaxis_label
    
    # add the lines
    l1 = p.line(forecasts.index, forecasts.values, color=c2, line_width = 2)
    l2 = p.line(forecasts_rs.index, forecasts_rs.values, color='coral', line_width = 2)
    l3 = p.line(true_forecasts.index, true_forecasts.values, color=c1,line_width=3)
    l4 = p.line(forecasts_1.index, forecasts_1.values, color='brown')
    l5 = p.line(forecasts_2.index, forecasts_2.values, color='goldenrod')
    l6 = p.line(forecasts_3.index, forecasts_3.values, color='purple')
    #l7 = p.line(forecasts_4.index, forecasts_4.values, color='red', line_width = 2)

    legend = Legend(items=[
    ('Forecasted_my_Stepwise'   , [l1]),
    #('Forecasted_my_Stepwise_force' , [l7]),
    ('Forecasted_my_RS' , [l2]),
    ('Observed' , [l3]),
    ('Forecasted_Theta' , [l4]),
    ('Forecasted_FPro' , [l5]),
    ('Forecasted_FX' , [l6]),], location=(0, -30))
    p.add_layout(legend, 'right')
    return p

Loading BokehJS ...

#### Training (Stepwise "full" auto)

In [109]:
from pyramid.arima import auto_arima

stepwise_fit = auto_arima(train_data, exogenous = exogenous_data,start_p=1, start_q=1, max_p=5, max_q=5, m=12,
                          start_P=0, max_d=4, max_D=2,max_P = 2, max_Q = 2, trace=False,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

stepwise_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   50
Model:               SARIMAX(0, 0, 1)   Log Likelihood                -412.034
Date:                Fri, 06 Jul 2018   AIC                            840.068
Time:                        13:01:06   BIC                            855.365
Sample:                             0   HQIC                           845.893
                                 - 50                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    103.5119    613.336      0.169      0.866   -1098.604    1305.628
x1             1.2798      0.184      6.971      0.000       0.920       1.640
x2             0.0022      0.033      0.067      0.947      -0.062       0.067
x3            -0.0006      0.092     -0.006      0.995      -0.180       0.179
x4             0.1520      0.087      1.753      0.080      -0.018       0.322
x5            -0.1609      0.225     -0.717      0.473      -0.601       0.279
ma.L1         -0.4209      0.173     -2.430      0.015      -0.760      -0.081
sigma2      9.119e+05   2.28e+05      4.006      0.000    4.66e+05    1.36e+06
===================================================================================
Ljung-Box (Q):                       28.87   Jarque-Bera (JB):                 0.55
Prob(Q):                              0.90   Prob(JB):                         0.76
Heteroskedasticity (H):               1.30   Skew:                             0.18
Prob(H) (two-sided):                  0.59   Kurtosis:                         3.37
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 7.25e+14. Standard errors may be unstable.
"""

In [110]:
#predict in sample = predicting on train sample
in_sample_preds = stepwise_fit.predict_in_sample(exogenous=exogenous_data)
#predicting on test(validate here)
next_validate = stepwise_fit.predict(n_periods=18,exogenous=new_exogenous)
print ("MAPE on train: ",mean_absolute_percentage_error(train_data, in_sample_preds))

MAPE on train:  17.52386770897459


In [111]:
show(plot_arima(train_data, in_sample_preds, 
                title="Original Series & In-sample Predictions", 
                c2='#FF0000', forecast_start=0))

#### Training (Stepwise force d = 1)

In [112]:
from pyramid.arima import auto_arima

stepwise_fit_force = auto_arima(train_data,exogenous=exogenous_data, start_p=1, start_q=1, max_p=5, max_q=5, m=12,
                          start_P=1, d=1, max_D=2,max_P = 2, max_Q = 2, trace=False, seasonal = True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

stepwise_fit_force.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   50
Model:               SARIMAX(0, 1, 2)   Log Likelihood                -406.485
Date:                Fri, 06 Jul 2018   AIC                            830.971
Time:                        13:01:24   BIC                            847.997
Sample:                             0   HQIC                           837.431
                                 - 50                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      8.0206     10.603      0.756      0.449     -12.762      28.803
x1             1.1695      0.224      5.225      0.000       0.731       1.608
x2            -0.0083      0.044     -0.188      0.851      -0.094       0.078
x3             0.0239      0.121      0.197      0.844      -0.213       0.261
x4             0.1093      0.114      0.955      0.339      -0.115       0.333
x5            -0.1032      0.285     -0.362      0.718      -0.662       0.456
ma.L1         -1.3164      0.326     -4.038      0.000      -1.955      -0.677
ma.L2          0.3366      0.212      1.590      0.112      -0.078       0.751
sigma2        1.2e+06   5.83e+05      2.060      0.039    5.81e+04    2.34e+06
===================================================================================
Ljung-Box (Q):                       30.09   Jarque-Bera (JB):                 0.99
Prob(Q):                              0.87   Prob(JB):                         0.61
Heteroskedasticity (H):               1.18   Skew:                             0.32
Prob(H) (two-sided):                  0.74   Kurtosis:                         2.71
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [113]:
in_sample_preds_force = stepwise_fit_force.predict_in_sample(exogenous = exogenous_data)
next_validate_force = stepwise_fit_force.predict(n_periods=18,exogenous=new_exogenous)
print ("MAPE on train:",mean_absolute_percentage_error(train_data, in_sample_preds_force))

MAPE on train: 19.40890686382415


In [114]:
show(plot_arima(train_data, in_sample_preds_force, 
                title="Original Series & In-sample Predictions", 
                c2='#FF0000', forecast_start=0))

###  Random search

In [115]:
rs_fit = auto_arima(train_data, exogenous=exogenous_data,start_p=1, start_q=1, max_p=5, max_q=5, m=12,
                    start_P=0, n_jobs=-1, max_d=4, max_D=2,max_P = 2, max_Q = 2, trace=False,
                    error_action='ignore',  # don't want to know if an order does not work
                    suppress_warnings=True,  # don't want convergence warnings
                    stepwise=False, random=True, random_state=42,  # we can fit a random search (not exhaustive)
                    n_fits=200)

rs_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                   50
Model:             SARIMAX(1, 0, 1)x(0, 0, 1, 12)   Log Likelihood                -411.293
Date:                            Fri, 06 Jul 2018   AIC                            842.586
Time:                                    13:02:03   BIC                            861.706
Sample:                                         0   HQIC                           849.867
                                             - 50                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    100.1083    635.346      0.158      0.875   -1145.147    1345.364
x1             1.2741      0.179      7.131      0.000       0.924       1.624
x2            -0.0019      0.032     -0.057      0.954      -0.065       0.061
x3             0.0199      0.105      0.190      0.850      -0.186       0.226
x4             0.1197      0.079      1.518      0.129      -0.035       0.274
x5            -0.0883      0.240     -0.369      0.712      -0.558       0.381
ar.L1          0.0435      0.541      0.080      0.936      -1.018       1.105
ma.L1         -0.4165      0.514     -0.811      0.418      -1.424       0.591
ma.S.L12      -0.2159      0.261     -0.827      0.408      -0.728       0.296
sigma2      9.109e+05      0.196   4.65e+06      0.000    9.11e+05    9.11e+05
===================================================================================
Ljung-Box (Q):                       26.34   Jarque-Bera (JB):                 0.29
Prob(Q):                              0.95   Prob(JB):                         0.86
Heteroskedasticity (H):               1.31   Skew:                             0.16
Prob(H) (two-sided):                  0.59   Kurtosis:                         2.80
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 1.01e+23. Standard errors may be unstable.
"""

In [118]:
in_sample_preds_rs = rs_fit.predict_in_sample(exogenous = exogenous_data)
next_validate_rs = rs_fit.predict(n_periods=18,exogenous = new_exogenous)
print ("MAPE on train",mean_absolute_percentage_error(train_data, in_sample_preds_rs))

MAPE on train 18.009181549411693


In [119]:
show(plot_arima(train_data, in_sample_preds_rs, 
                title="Original Series & In-sample Predictions", 
                c2='#FF0000', forecast_start=0))

In [120]:
show(plot_arima_comparison(validate_data,next_validate,next_validate_rs,
                           forecast_Theta,forecast_FPro,forecast_FX,next_validate_force))

In [121]:
print ('MAPE (Stepwise)      :', mean_absolute_percentage_error(validate_data,next_validate))
print ('MAPE (Stepwise force):', mean_absolute_percentage_error(validate_data,next_validate_force))
print ('MAPE (RS)            :', mean_absolute_percentage_error(validate_data,next_validate_rs))
print ('MAPE (Theta)         :', mean_absolute_percentage_error(validate_data,forecast_Theta))
print ('MAPE (FPro)          :', mean_absolute_percentage_error(validate_data, forecast_FPro))
print ('MAPE (FX)            :', mean_absolute_percentage_error(validate_data,forecast_FX))

MAPE (Stepwise)      : 17.930912596158667
MAPE (Stepwise force): 22.513706183987896
MAPE (RS)            : 18.645117343575002
MAPE (Theta)         : 17.616221219670344
MAPE (FPro)          : 21.12357924200018
MAPE (FX)            : 20.037732943190264
